In [1]:
#Eclipse
import re
with open('eclipse_mod.csv', 'w') as f:
    for line in open('EclipseBugRepository.xml'):
        if re.match(r'  <bug id="(\d*)" opendate="(.*)" fixdate="(.*)">', line, re.I):
            bug = re.match(r'  <bug id="(\d*)" opendate="(.*)" fixdate="(.*)">', line, re.I)
            bugid = bug.group(1)
#            opendate = bug.group(2)
#            fixdate = bug.group(3)
            f.write(bugid + '\n')

import MySQLdb
import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf8')
eclipse_bugid = pd.read_csv('eclipse_mod.csv', names = ['bug_id'])
conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')
df = pd.read_sql('select bug_id, short_desc from bugs', conn)
eclipse_bug = pd.merge(eclipse_bugid, df, on='bug_id')
eclipse_bug['index'] = eclipse_bug['bug_id']
eclipse_bug.set_index('index', inplace=True)



#eclipse_original = pd.read_pickle('../dq-original/eclipse_original.pkl')
activities = pd.read_sql('SELECT * FROM bugs_activity where fieldid in (2) ' , conn)
activities = pd.merge(eclipse_bugid, activities, on='bug_id')
activities.sort('bug_when', ascending=False, inplace=True)
eclipse_original = eclipse_bug.copy()

for rownum in range(0, activities.shape[0]):
    row = activities.iloc[rownum]
    bug_id = row['bug_id']
    removed = row['removed']
    try:
        eclipse_original.ix[bug_id, 'short_desc'] = removed
    except Exception:
        pass



import cgi
diff = 0
flag = False
bugline = ''
with open('EclipseBugRepository_initial.xml', 'w') as f:
    for line in open('EclipseBugRepository.xml'):
        if re.match(r' .*', line):
            if re.match(r'  <bug id="(\d*)" opendate="(.*)" fixdate="(.*)">', line, re.I):
                bug = re.match(r'  <bug id="(\d*)" opendate="(.*)" fixdate="(.*)">', line, re.I)
                bugid = int(bug.group(1))
                bugline = line
                flag = False
            elif re.match(r'      <summary>(.*)</summary>', line, re.I):
                bug = re.match(r'      <summary>(.*)</summary>', line, re.I)
                mod_summary = bug.group(1)
                initial_summary = cgi.escape(eclipse_original.ix[bugid, 'short_desc'])
                eclipse_bug.ix[bugid, 'initial_summary'] = initial_summary
                if mod_summary != initial_summary:
                    diff = diff + 1
                    flag = True
                    f.write(bugline)
                    f.write('    <buginformation>\n')
                    f.write('      <summary>' + initial_summary + '</summary>\n')
            elif flag:
                f.write(line)
        else:
            f.write(line)
print diff #Eclipsediff=953